In [1]:
using JSON
using JuMP

poaDictAprPM = Dict()

using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM_ext

In [2]:
include("../Julia_files/load_network_uni_class.jl")

load_ta_network (generic function with 2 methods)

In [3]:
#load OD pair-route incidence
odPairRoute = readstring("od_pair_route_incidence_MA_ext.json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readstring("link_route_incidence_MA_ext.json");
linkRoute = JSON.parse(linkRoute);
#load OD pair labels
odPairLabel = readstring("od_pair_label_dict_MA_refined_ext.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readstring("od_pair_label_dict__MA_refined_ext.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readstring("link_label_dict_MA_ext.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readstring("link_label_dict_MA__ext.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readstring("node_link_incidence_MA_ext.json");
nodeLink = JSON.parse(nodeLink);

ta_data = load_ta_network("MA");

In [4]:
odPairLabel_

Dict{String,Any} with 462 entries:
  "306" => Any[15,12]
  "153" => Any[8,6]
  "1"   => Any[1,2]
  "54"  => Any[3,13]
  "101" => Any[5,18]
  "371" => Any[18,14]
  "41"  => Any[2,21]
  "65"  => Any[4,2]
  "447" => Any[22,6]
  "335" => Any[16,21]
  "362" => Any[18,5]
  "326" => Any[16,11]
  "299" => Any[15,5]
  "168" => Any[8,22]
  "403" => Any[20,4]
  "159" => Any[8,13]
  "228" => Any[11,19]
  "332" => Any[16,18]
  "190" => Any[10,1]
  "270" => Any[13,19]
  "227" => Any[11,18]
  "223" => Any[11,14]
  "453" => Any[22,12]
  "88"  => Any[5,4]
  "297" => Any[15,3]
  ⋮     => ⋮

In [5]:
function POA_MA_Apr_PM_ext(day)

    capacity = ta_data.capacity;
    free_flow_time = ta_data.free_flow_time;

    demandsDict = readstring("../08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_PM/results/demandsDictFixed$(day).json");
    demandsDict = JSON.parse(demandsDict);

    # demands = demandsDict["$(length(demandsDict)-1)"]

    demands = demandsDict

    odPairLabel_;

    demandsVec = zeros(462)

    for i = 1:length(demandsVec)
        demandsVec[i] = demands["($(odPairLabel_["$i"][1]),$(odPairLabel_["$i"][2]))"]
    end

    demandsVec

    for key=keys(odPairRoute)
        if contains(key, "462-")
            println(key)
        end
    end

    linkRoute;

    deg = 6
    c = 0.4
    lam = 100.0

    coeffs_dict_Apr_PM_ = readstring("../temp_files/coeffs_dict_Apr_PM.json")
    coeffs_dict_Apr_PM_ = JSON.parse(coeffs_dict_Apr_PM_)
    fcoeffs = coeffs_dict_Apr_PM_["($(deg),$(c),$(lam),1)"]

    # fcoeffs = [1, 0, 0, 0, .15]

    # m = Model(solver=GurobiSolver(OutputFlag=false))
    m = Model()

    numLinks = 74
    numRoute = 4614
    numOD = 462

    @variable(m, linkFlow[1:numLinks])
    @variable(m, pathFlow[1:numRoute])

    pathFlowSum = Dict()

    for i=1:numOD
        pathFlowSum[i] = 0
        for j=1:numRoute
            if "$(i)-$(j)" in keys(odPairRoute)
                pathFlowSum[i] += pathFlow[j]
            end
        end
        @constraint(m, pathFlowSum[i] == demandsVec[i])
    end

    pathFlowLinkSum = Dict()

    for a=1:numLinks
        pathFlowLinkSum[a] = 0
        for j=1:numRoute
            if "$(a)-$(j)" in keys(linkRoute)
                pathFlowLinkSum[a] += pathFlow[j]
            end
        end
        @constraint(m, pathFlowLinkSum[a] == linkFlow[a])
    end

    for j=1:numRoute
        @constraint(m, pathFlow[j] >= 0)
    end

    @NLexpression(m, f, sum{free_flow_time[a] * fcoeffs[1] * linkFlow[a] + 
        free_flow_time[a] * fcoeffs[2] * linkFlow[a]^2 / capacity[a] +
        free_flow_time[a] * fcoeffs[3] * linkFlow[a]^3 / capacity[a]^2 +
        free_flow_time[a] * fcoeffs[4] * linkFlow[a]^4 / capacity[a]^3 +
        free_flow_time[a] * fcoeffs[5] * linkFlow[a]^5 / capacity[a]^4 +
        free_flow_time[a] * fcoeffs[6] * linkFlow[a]^6 / capacity[a]^5 +
        free_flow_time[a] * fcoeffs[7] * linkFlow[a]^7 / capacity[a]^6, a = 1:numLinks})

    @NLobjective(m, Min, f)

    solve(m)

    println(getvalue(linkFlow))

    getobjectivevalue(m)

    flows = Dict()

    for i = 1:length(ta_data.start_node)
        key = (ta_data.start_node[i], ta_data.end_node[i])
        flows[key] = getvalue(linkFlow)[i]
    end

    flows

    # getvalue(linkFlow)

    # getobjectivevalue(m)

    flow_user = GLS_Apr_weekday_PM_ext.x_

    function socialObj(linkFlowVec)
        objVal = sum([free_flow_time[a] * fcoeffs[1] * linkFlowVec[a] + 
            free_flow_time[a] * fcoeffs[2] * linkFlowVec[a]^2 / capacity[a] + 
            free_flow_time[a] * fcoeffs[3] * linkFlowVec[a]^3 / capacity[a]^2 + 
            free_flow_time[a] * fcoeffs[4] * linkFlowVec[a]^4 / capacity[a]^3 + 
            free_flow_time[a] * fcoeffs[5] * linkFlowVec[a]^5 / capacity[a]^4 + 
            free_flow_time[a] * fcoeffs[6] * linkFlowVec[a]^6 / capacity[a]^5 + 
            free_flow_time[a] * fcoeffs[7] * linkFlowVec[a]^7 / capacity[a]^6 for a = 1:numLinks])
        return objVal
    end
    
    println(socialObj(flow_user[:, day])/getobjectivevalue(m))
    
    return socialObj(flow_user[:, day])/getobjectivevalue(m)
end

POA_MA_Apr_PM_ext (generic function with 1 method)

In [6]:
function POA_MA_Apr_PM_ext_alt(day)

    capacity = ta_data.capacity;
    free_flow_time = ta_data.free_flow_time;

    demandsDict = readstring("../08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_PM/results/demandsDictFixed$(day).json");
    demandsDict = JSON.parse(demandsDict);

    # demands = demandsDict["$(length(demandsDict)-1)"]

    demands = demandsDict
    
    demands_ = Dict()

    for key in keys(demands)
        key_ = (parse(Int, split(split(key, ',')[1], '(')[2]),parse(Int, split(split(key, ',')[2], ')')[1]))
        demands_[key_] = demands[key]
    end

#     odPairLabel_;

    demandsVec = zeros(462)

    for i = 1:length(demandsVec)
        demandsVec[i] = demands["($(odPairLabel_["$i"][1]),$(odPairLabel_["$i"][2]))"]
    end

#     demandsVec

    for key=keys(odPairRoute)
        if contains(key, "462-")
            println(key)
        end
    end

#     linkRoute;

    deg = 8
    c = 0.5
    lam = 10000.0

    coeffs_dict_Apr_PM_ = readstring("../temp_files/coeffs_dict_Apr_PM.json")
    coeffs_dict_Apr_PM_ = JSON.parse(coeffs_dict_Apr_PM_)
    fcoeffs = coeffs_dict_Apr_PM_["($(deg),$(c),$(lam),1)"]

    # fcoeffs = [1, 0, 0, 0, .15]
    
    # m = Model(solver=GurobiSolver(OutputFlag=false))
    m = Model()

    numLinks = 74  # number of links
    numNodes = 22  # number of nodes

    @variable(m, linkFlow[1:numLinks])

    @variable(m, odLinkFlow[keys(demands_), 1:numLinks])
    for (s,t) = keys(demands_)
        for k = 1:numLinks
            @constraint(m, odLinkFlow[(s,t), k] >= 0)
        end
    end

    for k = 1:numLinks
        linkFlowPartial = sum([odLinkFlow[(s,t), k] for (s,t) = keys(demands_)])
        @constraint(m, linkFlowPartial == linkFlow[k])
    end

    for l = 1:numNodes
        for (s,t) = keys(demands_)
            if s != t
                odLinkFlowPartial = sum([nodeLink["$(l-1)-$(k-1)"] * odLinkFlow[(s,t), k] for k = 1:numLinks])
                if (l == s)
                    @constraint(m, odLinkFlowPartial + demands_[(s,t)] == 0)
                elseif (l == t)
                    @constraint(m, odLinkFlowPartial - demands_[(s,t)] == 0)
                else
                    @constraint(m, odLinkFlowPartial == 0)
                end
            end
        end
    end

    @NLexpression(m, f, sum{free_flow_time[a] * fcoeffs[1] * linkFlow[a] + 
        free_flow_time[a] * fcoeffs[2] * linkFlow[a]^2 / capacity[a] +
        free_flow_time[a] * fcoeffs[3] * linkFlow[a]^3 / capacity[a]^2 +
        free_flow_time[a] * fcoeffs[4] * linkFlow[a]^4 / capacity[a]^3 +
        free_flow_time[a] * fcoeffs[5] * linkFlow[a]^5 / capacity[a]^4 +
        free_flow_time[a] * fcoeffs[6] * linkFlow[a]^6 / capacity[a]^5 +
        free_flow_time[a] * fcoeffs[7] * linkFlow[a]^7 / capacity[a]^6, a = 1:numLinks})

    @NLobjective(m, Min, f)

    solve(m)

    println(getvalue(linkFlow))

    getobjectivevalue(m)

    flows = Dict()

    for i = 1:length(ta_data.start_node)
        key = (ta_data.start_node[i], ta_data.end_node[i])
        flows[key] = getvalue(linkFlow)[i]
    end

#     flows

    # getvalue(linkFlow)

    # getobjectivevalue(m)

    flow_user = GLS_Apr_weekday_PM_ext.x_

    function socialObj(linkFlowVec)
        objVal = sum([free_flow_time[a] * fcoeffs[1] * linkFlowVec[a] + 
            free_flow_time[a] * fcoeffs[2] * linkFlowVec[a]^2 / capacity[a] + 
            free_flow_time[a] * fcoeffs[3] * linkFlowVec[a]^3 / capacity[a]^2 + 
            free_flow_time[a] * fcoeffs[4] * linkFlowVec[a]^4 / capacity[a]^3 + 
            free_flow_time[a] * fcoeffs[5] * linkFlowVec[a]^5 / capacity[a]^4 + 
            free_flow_time[a] * fcoeffs[6] * linkFlowVec[a]^6 / capacity[a]^5 + 
            free_flow_time[a] * fcoeffs[7] * linkFlowVec[a]^7 / capacity[a]^6 for a = 1:numLinks])
        return objVal
    end
    
    println(socialObj(flow_user[:, day])/getobjectivevalue(m))
    
    return socialObj(flow_user[:, day])/getobjectivevalue(m)
end

POA_MA_Apr_PM_ext_alt (generic function with 1 method)

In [7]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]
# week_day_Apr_list = [11]

21-element Array{Int64,1}:
  2
  3
  4
  5
  6
  9
 10
 11
 12
 13
 16
 17
 18
 19
 20
 23
 24
 25
 26
 27
 30

In [8]:
# for day in week_day_Apr_list
#     poaDictAprPM[day] = POA_MA_Apr_PM_ext(day)
# end

In [9]:
for day in week_day_Apr_list
    poaDictAprPM[day] = POA_MA_Apr_PM_ext_alt(day)
end

462-329

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   788026
Number of nonzeros in inequality constraint Jacobian.:    35816
Number of nonzeros in Lagrangian Hessian.............:       74

Total number of variables............................:    35890
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number

1.5291460283991516
462-329
This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   788026
Number of nonzeros in inequality constraint Jacobian.:    35816
Number of nonzeros in Lagrangian Hessian.............:       74

Total number of variables............................:    35890
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:    10238
Total number of inequality constraints...............:    35816
        inequality constraints with only lower bounds:    35816
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) a

[2846.32,1102.29,304.939,1198.41,-3.41718e-6,2.25224,3628.24,1566.51,862.306,1794.78,371.762,264.806,-3.58069e-6,-3.56678e-6,3580.76,3566.29,1975.74,1880.91,2397.47,2489.0,1248.14,1927.78,2340.93,2380.32,-3.28805e-6,-3.4064e-6,3973.52,3980.59,2700.42,3173.17,4012.09,4020.11,110.985,170.271,100.412,253.497,2239.26,2286.12,4343.0,4235.6,2382.37,3131.63,-3.89774e-6,-3.88022e-6,2208.11,1957.88,2185.84,2028.58,510.996,110.18,513.879,410.434,1396.23,1815.44,370.112,76.5377,215.227,110.993,1672.7,611.72,-3.6819e-6,-3.87743e-6,3610.28,2981.03,-3.6299e-6,-3.67929e-6,2649.65,2363.29,2204.24,1383.8,808.455,338.905,1860.35,1262.69]
1.441666180800157
462-329
This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   788026
Number of nonzeros in inequality constraint Jacobian.:    35816
Number of nonzeros in Lagrangian Hessian.............:       74

Total numb

[2944.44,1016.19,390.704,1243.38,-9.10785e-7,4.52374,3612.21,1422.83,1092.56,1922.54,485.862,346.613,-1.38873e-6,-1.31069e-6,3562.7,3462.1,2028.09,1857.0,2871.92,2803.91,1409.76,1883.45,2822.54,2814.43,-5.71895e-7,-8.42844e-7,4128.94,3987.66,2700.65,3105.04,4197.66,4032.35,232.533,148.554,85.8087,377.134,2301.44,2398.51,4785.7,4195.23,2564.53,3235.73,-2.23977e-6,-2.19266e-6,2180.45,1940.21,2245.77,2161.31,488.206,114.688,580.024,580.286,1534.59,1949.36,491.873,91.7238,183.878,128.357,1919.61,727.658,-1.63757e-6,-2.18734e-6,3516.17,3050.23,-1.45408e-6,-1.65354e-6,3056.61,2654.37,2516.04,1577.12,1026.14,409.589,1976.18,1574.07]
1.7803258190949882
462-329
This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   788026
Number of nonzeros in inequality constraint Jacobian.:    35816
Number of nonzeros in Lagrangian Hessian.............:       74

Tot

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   788026
Number of nonzeros in inequality constraint Jacobian.:    35816
Number of nonzeros in Lagrangian Hessian.............:       74

Total number of variables............................:    35890
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:    10238
Total number of inequality constraints...............:    35816
        inequality constraints with only lower bounds:    35816
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

...

[2192.57,864.895,630.867,1319.94,-3.38806e-6,-3.30105e-6,3142.06,1132.94,913.116,1914.46,374.389,216.869,-3.52555e-6,-3.59255e-6,4246.25,4240.14,2016.65,1942.76,2158.96,2735.39,1044.38,1935.6,1908.0,2156.67,-2.79381e-6,-3.43062e-6,3853.29,4195.2,3107.01,3029.32,3804.26,4155.09,131.338,69.8616,53.7235,255.323,1636.65,1893.72,4126.82,4227.81,2645.6,3027.59,-3.88693e-6,-3.89536e-6,2106.08,1986.75,1678.28,1618.73,294.495,53.7913,375.582,319.935,1430.72,1670.48,330.229,60.4345,107.858,94.0019,1420.97,564.731,-3.71883e-6,-3.87049e-6,3652.13,3035.05,-3.64148e-6,-3.67031e-6,2745.26,2130.69,2065.28,1323.81,933.159,270.959,1583.22,1189.81]
1.3685247512589025
462-329
This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   788026
Number of nonzeros in inequality constraint Jacobian.:    35816
Number of nonzeros in Lagrangian Hessian.............:       74


[2257.12,792.517,0.0233286,1261.6,-1.06497e-6,-3.49411e-7,2329.79,837.374,66.0865,1456.19,184.437,41.0923,-1.47098e-6,-1.33423e-6,3217.28,3203.68,1847.37,1847.59,525.754,1195.75,121.574,1391.27,542.209,1245.08,-9.15762e-7,-9.09747e-7,3422.07,3972.85,1897.16,2628.64,3446.76,3972.85,55.181,34.5156,42.037,76.4582,528.919,1196.96,3540.47,3988.61,1887.56,2692.11,-2.23796e-6,-2.2305e-6,2013.68,2015.56,1321.01,1321.13,96.2629,35.3344,162.86,173.069,1362.91,1382.85,143.381,37.1699,59.3505,60.1828,587.506,305.652,-1.76628e-6,-2.19027e-6,3502.41,2736.71,-1.5177e-6,-1.64798e-6,2600.2,2585.25,1630.14,1311.74,470.207,213.396,1931.17,1035.64]
1.431866660900548
462-329
This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   788026
Number of nonzeros in inequality constraint Jacobian.:    35816
Number of nonzeros in Lagrangian Hessian.............:       74

T

[1748.19,953.48,1685.68,1524.66,-8.4289e-7,-7.7612e-7,3431.05,1594.03,2280.07,2404.31,663.605,495.723,-1.27386e-6,-1.31353e-6,4189.75,4362.05,2363.96,2264.17,3965.87,4122.0,2378.73,2252.74,3877.15,3967.62,458.551,-8.50939e-7,3860.06,4304.0,3937.31,3125.65,4295.17,4269.93,195.324,426.026,358.56,790.772,3253.84,3413.5,5061.78,4984.93,3213.08,3015.24,-2.15992e-6,-2.26163e-6,3441.17,2842.22,2604.66,2454.06,927.801,276.003,807.677,592.66,2349.21,2374.17,557.303,100.175,262.004,143.562,2670.06,2521.39,-1.73037e-6,-2.0641e-6,3681.56,3224.98,-1.54792e-6,-1.58669e-6,3427.08,2416.43,2297.06,1429.0,1134.13,1000.76,2030.35,1483.51]
3.0683833959562077
462-329
This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   788026
Number of nonzeros in inequality constraint Jacobian.:    35816
Number of nonzeros in Lagrangian Hessian.............:       74

Total num

[2843.23,914.327,0.64219,1090.84,-9.60966e-7,5.77703,3210.12,1166.78,459.678,1641.61,425.65,209.528,-1.39109e-6,-1.32903e-6,3561.8,3498.28,1878.67,1841.22,1669.02,2095.06,774.707,1560.43,1543.37,2096.96,-6.10059e-7,-8.83471e-7,3906.42,4039.0,2458.48,2843.42,4054.61,4036.13,194.703,96.117,82.1176,195.473,1271.92,1864.75,4352.49,4126.58,2320.39,2890.49,-2.23363e-6,-2.20796e-6,2043.11,1968.64,1681.36,1680.85,250.33,87.3307,448.849,542.197,1481.07,1642.68,478.303,77.738,140.747,113.243,1273.89,585.056,-1.68212e-6,-2.1785e-6,3413.57,3073.39,-1.49822e-6,-1.62047e-6,2876.91,2569.79,2053.74,1397.25,814.477,278.354,1802.35,1360.91]
1.9553435749352075
462-329
This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   788026
Number of nonzeros in inequality constraint Jacobian.:    35816
Number of nonzeros in Lagrangian Hessian.............:       74

Total 

[2843.23,914.327,0.64219,1090.84,-9.60966e-7,5.77703,3210.12,1166.78,459.678,1641.61,425.65,209.528,-1.39109e-6,-1.32903e-6,3561.8,3498.28,1878.67,1841.22,1669.02,2095.06,774.707,1560.43,1543.37,2096.96,-6.10059e-7,-8.83471e-7,3906.42,4039.0,2458.48,2843.42,4054.61,4036.13,194.703,96.117,82.1176,195.473,1271.92,1864.75,4352.49,4126.58,2320.39,2890.49,-2.23363e-6,-2.20796e-6,2043.11,1968.64,1681.36,1680.85,250.33,87.3307,448.849,542.197,1481.07,1642.68,478.303,77.738,140.747,113.243,1273.89,585.056,-1.68212e-6,-2.1785e-6,3413.57,3073.39,-1.49822e-6,-1.62047e-6,2876.91,2569.79,2053.74,1397.25,814.477,278.354,1802.35,1360.91]
1.155409799315816
462-329
This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   788026
Number of nonzeros in inequality constraint Jacobian.:    35816
Number of nonzeros in Lagrangian Hessian.............:       74

Total n

In [15]:
outfile = open("./results/poaDictAprPM.json", "w")

JSON.print(outfile, poaDictAprPM)

close(outfile)

In [16]:
poaDictAprPM

Dict{Any,Any} with 21 entries:
  30 => 1.48607
  2  => 1.52915
  16 => 1.48854
  11 => 1.62226
  26 => 1.93907
  9  => 1.20244
  25 => 1.95534
  10 => 2.33568
  19 => 1.43187
  17 => 1.52292
  6  => 1.40174
  24 => 1.95793
  4  => 1.44167
  3  => 1.49783
  5  => 1.78033
  20 => 1.01136
  23 => 1.53419
  13 => 1.59363
  27 => 1.15541
  12 => 1.46379
  18 => 1.36852